# Real-World Data Tweet Analysis

Analysis of Real-World Data (RWD) found in tweets.

In [1]:
import os
import transformers
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import xgboost
import numpy as np
import pandas as pd

D:\My Documents\Visual Studio projects\Projects\Python\torchenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. ALBERT

### 1.1. Task setup

In [3]:
task = "cola"
model_name = "distilbert-base-uncased"
batch_size = 1
metric = load_metric('glue', task)

D:\Temp\ipykernel_4932\1045784386.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', task)


### 1.2. Load dataset

In [ ]:
dataset = load_dataset("csv", data_files = { "train": "data/training.csv",
                                             "test": "data/test.csv" },
                      features = datasets.Features({ "Message": datasets.Value("string"),
                                                   "label": datasets.ClassLabel(names = [0, 1])}))

In [6]:
def preprocess_function(examples):
    return tokenizer(examples[text_key], truncation = True)

In [7]:
text_key = "Message"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

In [9]:
dataset["train"].features

{'Message': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=[0, 1], names_file=None, id=None)}

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched = True)

In [11]:
num_labels = 3 if task.startswith("mlni") else 1 if task == "stsb" else 2

In [ ]:
modelCheckpoint = f"models/{model_name}"

### 1.3. Finetune a model

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [14]:
metric_name = "f1"

args = TrainingArguments(
    "test-glue",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    num_train_epochs = 5,
    weight_decay = 0.01,
    seed = 1194,
)

In [15]:
encoded_dataset['train'] = encoded_dataset['train'].rename_column("Message", "sentence")
encoded_dataset['test'] = encoded_dataset['test'].rename_column("Message", "sentence")

In [16]:
encoded_dataset["train"].features

{'sentence': Value(dtype='string', id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': ClassLabel(num_classes=2, names=[0, 1], names_file=None, id=None)}

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset = encoded_dataset["train"],
)

In [19]:
trainer.train()

Step,Training Loss
500,0.809900
1000,0.491400
1500,0.224300
2000,0.071700
2500,0.045800


TrainOutput(global_step=2530, training_loss=0.32474020032190054, metrics={'train_runtime': 174.4835, 'train_samples_per_second': 14.5, 'total_flos': 54510751841400.0, 'epoch': 5.0})

In [20]:
model.save_pretrained(modelCheckpoint)

### 1.4. Evaluate a finetuned model

Load a model and evaluate a saved model.

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(modelCheckpoint, num_labels = num_labels)

In [22]:
trainer_test = Trainer(model, tokenizer = tokenizer)

In [23]:
pred_prob, _, _ = trainer_test.predict(encoded_dataset["test"])

In [24]:
y_pred = np.argmax(pred_prob, axis = 1)

In [ ]:
def print_evaluation_metrics(label, y_true, y_pred):
    print("{} prediction metrics\nF1: {:.4f}\nPrecision: {:.4f}\nRecall: {:.4f}".format(
        label,
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred)
    ))

In [ ]:
print_evaluation_metrics(label = "ALBERT", y_true = encoded_dataset["test"]["label"], y_pred = y_pred)

## 2. TF-IDF vectorization

Vectorize data to use for subsequent models.

In [ ]:
df_train = pd.read_csv("data/training.csv")
df_test = pd.read_csv("data/test.csv")

In [ ]:
vectorizer = TfidfVectorizer(min_df = 2)

In [ ]:
data_train = vectorizer.fit_transform(df_train["Message"].values)
data_test = vectorizer.transform(df_test["Message"].values)
labels_train, labels_test = data_train["label"], data_test["label"]

## 2. SVM

In [4]:
svm = SVC()

In [ ]:
svm.fit(data_train, labels_train)

In [ ]:
svm_pred = svm.predict(data_test)

In [ ]:
svm_pred_binary = [int(x >= 0.5) for x in svm_pred]

In [ ]:
print_evaluation_metrics(label = "SVM", y_true = labels_test, y_pred = svm_pred_binary)

## 3. Random forest

In [ ]:
rf = RandomForestClassifier(n_estimators = 500)

In [ ]:
rf.fit(data_train, labels_train)

In [ ]:
rf_pred = rf.predict(data_test)

In [ ]:
rf_pred_binary = [int(x >= 0.5) for x in rf_pred]

In [ ]:
print_evaluation_metrics(label = "Random forest", y_true = labels_test, y_pred = rf_pred_binary)

## 4. XGBoost

In [ ]:
xgb = xgboost.XGBClassifier()

In [ ]:
xgb.fit(data_train, labels_train)

In [ ]:
xgb_pred = xgb.predict(data_test)

In [ ]:
xgb_pred_binary = [int(x >= 0.5) for x in xgb_pred]

In [ ]:
print_evaluation_metrics(label = "XGBoost", y_true = labels_test, y_pred = xgb_pred_binary)

## 5. Ensemble

In [ ]:
ensemble = VotingClassifier([svm, rf, xgb])

In [ ]:
ens_pred = ensemble.predict(data_test)

In [ ]:
print_evaluation_metrics(label = "Ensemble", y_true = labels_test, y_pred = ens_pred)